## Homework 2

In [62]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

### Q1. Embedding the query

In [2]:
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [3]:
query = 'I just discovered the course. Can I join now?'
query_emb = next(model.embed(query))
query_emb.shape

(512,)

In [4]:
query_emb.mean(), query_emb.dot(query_emb), query_emb @ query_emb

(np.float64(-0.0007128863462480066),
 np.float64(1.0000000000000002),
 np.float64(1.0000000000000002))

In [5]:
import numpy as np
np.linalg.norm(query_emb)

np.float64(1.0)

In [6]:
query_emb.min()

np.float64(-0.11726373885183883)

### Q2. Cosine similarity with another vector

In [7]:
doc = 'Can I still join the course after the start date?'
doc_emb = emb = next(model.embed(doc))

In [8]:
doc_emb @ query_emb

np.float64(0.9008528895674548)

### Q3. Ranking by cosine

In [9]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [10]:
len(documents)

5

In [12]:
texts = [d['text'] for d in documents]
texts

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'You can start by instal

In [15]:
doc_embs = list(model.embed(texts))

In [16]:
[de @ de for de in doc_embs]

[np.float64(1.0),
 np.float64(1.0000000000000002),
 np.float64(1.0),
 np.float64(1.0000000000000002),
 np.float64(1.0000000000000002)]

In [17]:
[de @ query_emb for de in doc_embs]

[np.float64(0.762968451872193),
 np.float64(0.8182378156620136),
 np.float64(0.8085397445747489),
 np.float64(0.7133078832064158),
 np.float64(0.7304499196411823)]

In [19]:
np.argmax([de @ query_emb for de in doc_embs])

np.int64(1)

or same via matrix:

In [23]:
np.array(doc_embs) @ query_emb

array([0.76296845, 0.81823782, 0.80853974, 0.71330788, 0.73044992])

In [24]:
(np.array(doc_embs) @ query_emb).argmax()

np.int64(1)

### Q4. Ranking by cosine, version two

In [27]:
full_texts = [d['question'] + ' ' + d['text'] for d in documents]

In [34]:
full_texts

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin t

In [31]:
doc_full_embs = list(model.embed(full_texts))

In [33]:
[de @ query_emb for de in doc_full_embs]

[np.float64(0.851454319443226),
 np.float64(0.8436594005975433),
 np.float64(0.8408287224005012),
 np.float64(0.7755157657626952),
 np.float64(0.8086007917931166)]

In [35]:
np.array(doc_full_embs) @ query_emb

array([0.85145432, 0.8436594 , 0.84082872, 0.77551577, 0.80860079])

In [36]:
(np.array(doc_full_embs) @ query_emb).argmax()

np.int64(0)

### Q5. Selecting the embedding model

In [41]:
from collections import Counter

In [42]:
Counter(model['dim'] for model in TextEmbedding.list_supported_models())

Counter({768: 15, 1024: 6, 384: 6, 512: 3})

In [43]:
min(model['dim'] for model in TextEmbedding.list_supported_models())

384

In [68]:
model_handle_small = 'BAAI/bge-small-en'

In [44]:
model_small = TextEmbedding(model_name=model_handle_small)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [57]:
next(model_small.embed(query)).shape

(384,)

### Q6. Indexing with qdrant (2 points)

In [58]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [59]:
len(documents)

375

In [60]:
collection_name = "hw2"

In [61]:
EMBEDDING_DIMENSIONALITY = next(model_small.embed(query)).shape[0]
EMBEDDING_DIMENSIONALITY

384

In [63]:
client = QdrantClient("http://localhost:6333")

In [64]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [65]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [67]:
len(documents)

375

In [70]:
model_handle_small

'BAAI/bge-small-en'

In [71]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle_small)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [73]:
points[:2]

[PointStruct(id=0, vector=Document(text='How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', model='BAAI/bge-small-en', options=None), payload={'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I 

In [74]:
collection_name

'hw2'

In [75]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [76]:
query

'I just discovered the course. Can I join now?'

In [80]:
res = client.query_points(
    collection_name=collection_name,
    query=models.Document(text=query, model=model_handle_small),
    limit=1,
    with_payload=True
)

res

QueryResponse(points=[ScoredPoint(id=14, version=2, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [83]:
res.points[0].score

0.87031734